In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .getOrCreate()

In [13]:
spark = SparkSession.builder.appName("종족별 랭킹").getOrCreate()
abyss = (spark.createDataFrame([
    (1, "천족", "Cat", 30), 
    (1, "천족", "Dog", 28), 
    (1, "마족", "Monkey", 28), 
    (1, "마족", "Cat", 24), 
    (1, "마족", "Dog", 10)
], ["Server", "Race", "Actor", "Point"]))
abyss.show(truncate=False)
abyss.createOrReplaceTempView("abyss")

# CASE WHEN key = 1 THEN 1 ELSE 2 END
light="천족_타이틀"
dark="마족_타이틀"
res = spark.sql("""
with ranked_table as ( select *, row_number() over (partition by server, race order by point desc) as rank from abyss)
select case when race = '천족' then '{}' else '{}' end as cat_name, * from ranked_table
""".format(light, dark))
res.show(truncate=False)

+------+----+------+-----+
|Server|Race|Actor |Point|
+------+----+------+-----+
|1     |천족|Cat   |30   |
|1     |천족|Dog   |28   |
|1     |마족|Monkey|28   |
|1     |마족|Cat   |24   |
|1     |마족|Dog   |10   |
+------+----+------+-----+

+-----------+------+----+------+-----+----+
|cat_name   |Server|Race|Actor |Point|rank|
+-----------+------+----+------+-----+----+
|천족_타이틀|1     |천족|Cat   |30   |1   |
|천족_타이틀|1     |천족|Dog   |28   |2   |
|마족_타이틀|1     |마족|Monkey|28   |1   |
|마족_타이틀|1     |마족|Cat   |24   |2   |
|마족_타이틀|1     |마족|Dog   |10   |3   |
+-----------+------+----+------+-----+----+

